# Human Motion Animation Generation Pipeline

This notebook implements a pipeline for generating human motion animations using:
- **Autoregressive Context Encoder**: Encodes motion context sequentially
- **Flow Matching Network**: Generates motion sequences using flow matching

**Compatible with MoMask input/output format:**
- Input: HumanML3D dim-263 feature vectors
- Output: Joint positions (nframe, 22, 3) → BVH files

In [ ]:
# Imports
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Add project root to path
sys.path.append(str(Path.cwd()))

from config import Config
from models import AutoregressiveContextEncoder, FlowMatchingNetwork
from utils import (
    load_humanml3d,
    preprocess_motion,
    create_dataloader,
    feature_to_joints,
    joints_to_bvh,
    save_bvh,
    save_joints,
    compute_metrics,
    visualize_motion,
)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load configuration
config = Config()

## Step 1: Data Preparation (HumanML3D)

Load and preprocess the HumanML3D dataset with dim-263 feature vectors.

In [ ]:
# TODO: Load HumanML3D dataset
# Expected format: dim-263 feature vectors, text-motion pairs
dataset_train = load_humanml3d(
    dataset_path=config.dataset_path,
    split="train",
    max_motion_length=config.max_motion_length,
)

dataset_val = load_humanml3d(
    dataset_path=config.dataset_path,
    split="val",
    max_motion_length=config.max_motion_length,
)

print(f"Train samples: {len(dataset_train)}")
print(f"Val samples: {len(dataset_val)}")

# TODO: Preprocess motion data
# Process dim-263 features, normalize, handle text descriptions
train_data = preprocess_motion(dataset_train, config)
val_data = preprocess_motion(dataset_val, config)

# TODO: Create data loaders
train_loader = create_dataloader(train_data, batch_size=config.batch_size, shuffle=True)

val_loader = create_dataloader(val_data, batch_size=config.batch_size, shuffle=False)

# TODO: Visualize sample data
sample_motion, sample_text = train_data[0]
print(f"Sample motion shape: {sample_motion.shape}")  # Expected: (seq_len, 263)
print(f"Sample text: {sample_text}")

# Convert to joints for visualization
sample_joints = feature_to_joints(sample_motion)  # (nframe, 22, 3)
print(f"Sample joints shape: {sample_joints.shape}")

## Step 2: Autoregressive Context Encoder

Initialize and test the autoregressive context encoder model.

In [ ]:
# TODO: Initialize Autoregressive Context Encoder
context_encoder = AutoregressiveContextEncoder(
    input_dim=config.motion_dim,  # 263
    hidden_dim=config.hidden_dim,
    num_layers=config.num_encoder_layers,
    max_seq_length=config.max_motion_length,
    bidirectional=config.bidirectional_gru,
).to(device)

print(
    f"Context Encoder parameters: {sum(p.numel() for p in context_encoder.parameters()):,}"
)

# TODO: Test forward pass
sample_batch_motion = torch.randn(
    config.batch_size, config.max_motion_length, config.motion_dim
).to(device)
sample_batch_text = ["A person is walking"] * config.batch_size

with torch.no_grad():
    context_output = context_encoder(sample_batch_motion, sample_batch_text)
    print(f"Context encoder output shape: {context_output.shape}")

## Step 3: Flow Matching Network

Initialize and test the flow matching network model.

In [ ]:
# TODO: Initialize Flow Matching Network
flow_matching_net = FlowMatchingNetwork(
    context_dim=context_encoder.output_dim,
    motion_dim=config.motion_dim,  # 263
    hidden_dim=config.hidden_dim,
    num_layers=config.num_flow_layers,
).to(device)

print(
    f"Flow Matching Network parameters: {sum(p.numel() for p in flow_matching_net.parameters()):,}"
)

# TODO: Test forward pass
with torch.no_grad():
    # Flow matching forward pass
    flow_output = flow_matching_net(context_output, sample_batch_motion)
    print(f"Flow matching output shape: {flow_output.shape}")

## Step 4: Training Loop

Set up training configuration, loss functions, and training loop.

In [ ]:
# TODO: Set up optimizers
optimizer_context = torch.optim.Adam(
    context_encoder.parameters(), lr=config.learning_rate
)

optimizer_flow = torch.optim.Adam(
    flow_matching_net.parameters(), lr=config.learning_rate
)


# TODO: Define loss functions
def compute_loss(predicted_motion, target_motion, context_output):
    """
    Compute training loss for flow matching.

    Args:
        predicted_motion: Generated motion from flow matching (batch, seq_len, 263)
        target_motion: Ground truth motion (batch, seq_len, 263)
        context_output: Context from autoregressive encoder

    Returns:
        loss: Scalar loss value
    """
    # TODO: Implement flow matching loss
    loss = nn.MSELoss()(predicted_motion, target_motion)
    return loss


# TODO: Training loop
def train_epoch(
    model_context, model_flow, train_loader, optimizer_context, optimizer_flow, device
):
    """
    Train for one epoch.
    """
    model_context.train()
    model_flow.train()

    total_loss = 0.0
    num_batches = 0

    for batch_idx, (motion, text) in enumerate(train_loader):
        # TODO: Move to device
        motion = motion.to(device)

        # TODO: Forward pass
        # 1. Encode context
        context = model_context(motion, text)

        # 2. Flow matching
        predicted_motion = model_flow(context, motion)

        # 3. Compute loss
        loss = compute_loss(predicted_motion, motion, context)

        # TODO: Backward pass
        optimizer_context.zero_grad()
        optimizer_flow.zero_grad()
        loss.backward()
        optimizer_context.step()
        optimizer_flow.step()

        total_loss += loss.item()
        num_batches += 1

        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}, Loss: {loss.item():.4f}")

    return total_loss / num_batches


# TODO: Validation loop
def validate(model_context, model_flow, val_loader, device):
    """
    Validate model performance.
    """
    model_context.eval()
    model_flow.eval()

    total_loss = 0.0
    num_batches = 0

    with torch.no_grad():
        for motion, text in val_loader:
            motion = motion.to(device)

            context = model_context(motion, text)
            predicted_motion = model_flow(context, motion)
            loss = compute_loss(predicted_motion, motion, context)

            total_loss += loss.item()
            num_batches += 1

    return total_loss / num_batches


# TODO: Training loop with checkpointing
num_epochs = config.num_epochs
best_val_loss = float("inf")

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    # Train
    train_loss = train_epoch(
        context_encoder,
        flow_matching_net,
        train_loader,
        optimizer_context,
        optimizer_flow,
        device,
    )
    print(f"Train Loss: {train_loss:.4f}")

    # Validate
    val_loss = validate(context_encoder, flow_matching_net, val_loader, device)
    print(f"Val Loss: {val_loss:.4f}")

    # TODO: Save checkpoint
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(
            {
                "context_encoder": context_encoder.state_dict(),
                "flow_matching_net": flow_matching_net.state_dict(),
                "epoch": epoch,
                "val_loss": val_loss,
            },
            config.checkpoint_dir / f"best_model_epoch_{epoch+1}.pt",
        )
        print(f"Saved best model (val_loss: {val_loss:.4f})")

## Step 5: Inference / Generation

Load trained models and generate motion sequences.

In [ ]:
# TODO: Load trained models
checkpoint_path = (
    config.checkpoint_dir / "best_model_epoch_X.pt"
)  # Update with actual path
checkpoint = torch.load(checkpoint_path, map_location=device)

context_encoder.load_state_dict(checkpoint["context_encoder"])
flow_matching_net.load_state_dict(checkpoint["flow_matching_net"])

context_encoder.eval()
flow_matching_net.eval()

print("Models loaded successfully")


# TODO: Generate motion sequences
def generate_motion(
    model_context, model_flow, text_prompt, motion_length=None, device="cuda"
):
    """
    Generate motion from text prompt.

    Args:
        model_context: Trained context encoder
        model_flow: Trained flow matching network
        text_prompt: Text description of desired motion
        motion_length: Desired motion length in frames (optional)
        device: Device to run on

    Returns:
        generated_motion: Generated motion as dim-263 features (seq_len, 263)
    """
    model_context.eval()
    model_flow.eval()

    with torch.no_grad():
        # TODO: Generate initial context from text
        # For now, use random initialization - will be replaced with text encoding
        if motion_length is None:
            motion_length = config.max_motion_length

        # TODO: Autoregressive generation with flow matching
        # 1. Initialize with context
        # 2. Iteratively generate using flow matching
        # 3. Return generated motion sequence

        # Placeholder: random generation for skeleton
        generated_motion = torch.randn(motion_length, config.motion_dim).to(device)

    return generated_motion.cpu().numpy()


# TODO: Generate from text prompts
text_prompts = [
    "A person is walking forward",
    "A person is running on a treadmill",
    "A person is dancing",
]

generated_motions = []
for text in text_prompts:
    motion = generate_motion(context_encoder, flow_matching_net, text, device=device)
    generated_motions.append(motion)
    print(f"Generated motion for: '{text}' - Shape: {motion.shape}")

# TODO: Convert to joint positions
generated_joints = []
for motion in generated_motions:
    joints = feature_to_joints(motion)  # (nframe, 22, 3)
    generated_joints.append(joints)
    print(f"Converted to joints - Shape: {joints.shape}")

## Step 6: Post-processing

Convert generated motions to BVH format and save files.

In [ ]:
# TODO: Create output directories
output_dir = Path(config.output_path) / "experiment_1"
joints_dir = output_dir / "joints"
animation_dir = output_dir / "animation"

joints_dir.mkdir(parents=True, exist_ok=True)
animation_dir.mkdir(parents=True, exist_ok=True)

# TODO: Convert joint positions to BVH and save
for idx, (joints, text) in enumerate(zip(generated_joints, text_prompts)):
    # Save joint positions as numpy file
    joints_file = joints_dir / f"motion_{idx:04d}.npy"
    np.save(joints_file, joints)
    print(f"Saved joints to {joints_file}")

    # Convert to BVH format
    bvh_data = joints_to_bvh(joints)

    # Save BVH file
    bvh_file = animation_dir / f"motion_{idx:04d}.bvh"
    save_bvh(bvh_data, bvh_file)
    print(f"Saved BVH to {bvh_file}")

    # TODO: Validate BVH structure
    is_valid = validate_bvh(bvh_file)
    print(f"BVH validation: {'Valid' if is_valid else 'Invalid'}")

print(f"\nAll outputs saved to {output_dir}")

## Step 7: Evaluation

Compute evaluation metrics and visualize generated motions.

In [ ]:
# TODO: Load ground truth motions for comparison
# For evaluation, compare generated motions with ground truth from validation set
val_motions = []
val_texts = []

for i in range(min(10, len(val_data))):  # Sample 10 validation motions
    motion, text = val_data[i]
    val_motions.append(motion)
    val_texts.append(text)

# Convert validation motions to joints
val_joints = [feature_to_joints(motion) for motion in val_motions]


# TODO: Compute evaluation metrics
def evaluate_generated_motions(
    generated_joints, ground_truth_joints, generated_texts, gt_texts
):
    """
    Compute evaluation metrics for generated motions.

    Metrics:
    - FID (Fréchet Inception Distance) - motion quality
    - Diversity - motion variety
    - R-Precision - text-motion alignment
    """
    # TODO: Implement metrics computation
    metrics = {
        "fid": 0.0,  # Placeholder
        "diversity": 0.0,  # Placeholder
        "r_precision": 0.0,  # Placeholder
    }
    return metrics


metrics = compute_metrics(generated_joints, val_joints, text_prompts, val_texts)
print("\nEvaluation Metrics:")
for metric_name, value in metrics.items():
    print(f"  {metric_name}: {value:.4f}")

# TODO: Visualize generated motions
for idx, (joints, text) in enumerate(zip(generated_joints, text_prompts)):
    print(f"\nVisualizing motion {idx+1}: '{text}'")
    visualize_motion(
        joints, title=text, save_path=animation_dir / f"vis_motion_{idx:04d}.png"
    )

# TODO: Compare with ground truth
print("\nComparing generated vs ground truth:")
for idx in range(min(3, len(generated_joints))):
    print(f"\nSample {idx+1}:")
    print(f"  Generated: '{text_prompts[idx]}'")
    print(f"  Ground Truth: '{val_texts[idx]}'")

    # Visualize comparison
    visualize_motion(
        generated_joints[idx],
        ground_truth=val_joints[idx],
        title=f"Generated vs GT - {idx+1}",
        save_path=animation_dir / f"comparison_{idx:04d}.png",
    )

print("\nEvaluation complete!")